<a href="https://colab.research.google.com/github/AnviTendulkar/Airline_Fare_Prediction/blob/master/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler ,FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import re
from datetime import datetime, timedelta
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, PowerTransformer ,OneHotEncoder
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import FeatureUnion, Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import mean_absolute_error

In [2]:
pd.set_option("display.max_columns" ,None)
pd.set_option("display.max_rows" ,None)

In [3]:
train_data = pd.read_excel(r'/content/drive/MyDrive/Project/Airline Fare  Prediction/Data_Train.xlsx')

In [4]:
train_data.head(3)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882


In [5]:
train_data.dropna(inplace=True)

In [6]:
X = train_data.loc[:,train_data.columns !='Price']
y =train_data['Price']

In [7]:
x_train ,x_test ,y_train ,y_test = train_test_split(X,y ,test_size=0.20)

In [8]:
def Arrival_Date_calculation(row):
    pattern = re.compile(r'(?:(\d+)h)?\s?(?:(\d+)m)?')
    match = pattern.match(row['Duration'])
    if match:
        hours = int(match.group(1)) if match.group(1) else 0
        minutes = int(match.group(2)) if match.group(2) else 0

    # Concatenate 'Date_of_Journey' and 'Dep_Time'
    date_of_journey = row['Date_of_Journey'] + ' ' + row['Dep_Time']

    # Convert the concatenated string to datetime
    journey_date_time = datetime.strptime(date_of_journey, "%d/%m/%Y %H:%M")

    # Calculate arrival date by adding 'Duration_hour' and 'Duration_minute' to 'Date_of_Journey'
    arrival_date = journey_date_time + timedelta(hours=hours, minutes=minutes)
    return arrival_date

def split_dep_time_in_hours_min(str1):
    lst = str1.split(':')
    return pd.Series({'Dep_Hour': lst[0], 'Dep_Minute': lst[1]})

In [9]:
###################
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd

class DateColumnExtractor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        temp_df = pd.DataFrame()
        X['Arrival_Date'] = X.apply(Arrival_Date_calculation, axis=1)
        temp_df['Arrival_Year'] = X['Arrival_Date'].dt.year
        temp_df['Arrival_Month'] = X['Arrival_Date'].dt.month
        temp_df['Arrival_Day'] = X['Arrival_Date'].dt.day
        temp_df['Arrival_Hour'] = X['Arrival_Date'].dt.hour
        temp_df['Arrival_Minute'] = X['Arrival_Date'].dt.minute

        temp = X['Date_of_Journey'].copy()
        temp = pd.to_datetime(temp ,format='%d/%m/%Y')
        # Extract year, month, and day into separate columns
        temp_df['Dep_Year'] = temp.dt.year
        temp_df['Dep_Month'] = temp.dt.month
        temp_df['Dep_Day'] = temp.dt.day

        temp_df[['Dep_Hour', 'Dep_Minute']] = X['Dep_Time'].apply(split_dep_time_in_hours_min)
        return temp_df

    def get_params(self, deep=True):
        return {}


In [17]:
#################################################

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin
import pandas as pd

# Assume the DateColumnExtractor class is defined as provided in your previous message

# Your DataFrame
df = x_train[['Airline', 'Date_of_Journey', 'Source', 'Destination', 'Dep_Time', 'Duration', 'Total_Stops']]

# Define the DateColumnExtractor
date_extractor = DateColumnExtractor()

# Define the ColumnTransformer
preprocessor = ColumnTransformer([
    ('one-hot', OneHotEncoder(drop='first', sparse_output=False, dtype='int16'), [0, 2, 3]),
    ('total_stop', FunctionTransformer(lambda x: x.replace({'1 stops': 1, '1 stop': 1, 'non-stop': 0, '2 stops': 2, '3 stops': 3, '4 stops': 4}),
                                      validate=False), [df.columns.get_loc('Total_Stops')]),
    ('date_extractor', date_extractor, ['Date_of_Journey', 'Dep_Time','Duration']),

], remainder='passthrough')

# Define the pipeline
# pipe_clf = Pipeline([
#     ('preprocessor', preprocessor),
#     ('scaling' ,StandardScaler()),
#     ('regression_model', LinearRegression())
# ])


clf = Pipeline([
    ('preprocessor', preprocessor),
    ('scaling' ,StandardScaler()),
    # ('regression_model', LinearRegression())
])
pipe_clf = Pipeline([
    ("clf" ,clf),
    ('regression_model', LinearRegression())
])

# Fit and transform the data
# transformed_data = pipe_clf.fit_transform(df) //if linear regression is used then fit method does not work
transformed_data = pipe_clf.fit(df,y_train)
# print(pd.DataFrame(transformed_data).head(3))



In [18]:
predictions = pipe_clf.predict(x_test)
from sklearn.metrics import mean_absolute_error
print("Model Accuracy: %.3f" % pipe_clf.score(x_test, y_test))
mae = mean_absolute_error(y_test, predictions)
print("The mean squared error (MSE) on test set: {:.4f}".format(mae))

Model Accuracy: 0.641
The mean squared error (MSE) on test set: 1986.2237


In [20]:
pipe_clf

Pipeline(steps=[('clf',
                 Pipeline(steps=[('preprocessor',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('one-hot',
                                                                   OneHotEncoder(drop='first',
                                                                                 dtype='int16',
                                                                                 sparse_output=False),
                                                                   [0, 2, 3]),
                                                                  ('total_stop',
                                                                   FunctionTransformer(func=<function <lambda> at 0x7c63e6e1cf70>),
                                                                   [6]),
                                                                  ('date_extractor',
                                                                   DateColumnExtractor(),
                                                                   ['Date_of_Journey',
                                                                    'Dep_Time',
                                                                    'Duration'])])),
                                 ('scaling', StandardScaler())])),
                ('regression_model', LinearRegression())])

In [19]:
# Get the names of the transformed columns
one_hot_columns = preprocessor.named_transformers_['one-hot'].get_feature_names_out(['Airline', 'Source', 'Destination'])
total_stop_columns = ['Total_Stops']  # You might need to update this based on your actual transformation

# Combine all transformed column names
transformed_columns = list(one_hot_columns) + total_stop_columns + ['Arrival_Year', 'Arrival_Month', 'Arrival_Day', 'Arrival_Hour', 'Arrival_Minute', 'Dep_Year', 'Dep_Month', 'Dep_Day', 'Dep_Hour', 'Dep_Minute']

# Concatenate the transformed data with the remaining columns and set column names
transformed_df = pd.DataFrame(clf.transform(df) ,columns = transformed_columns)

# Print the transformed DataFrame with column names
print(transformed_df.head(3))


   Airline_Air India  Airline_GoAir  Airline_IndiGo  Airline_Jet Airways  \
0          -0.445297      -0.137254       -0.487717             1.340884   
1           2.245694      -0.137254       -0.487717            -0.745777   
2          -0.445297      -0.137254       -0.487717             1.340884   

   Airline_Jet Airways Business  Airline_Multiple carriers  \
0                     -0.021641                  -0.354088   
1                     -0.021641                  -0.354088   
2                     -0.021641                  -0.354088   

   Airline_Multiple carriers Premium economy  Airline_SpiceJet  \
0                                  -0.034229         -0.290027   
1                                  -0.034229         -0.290027   
2                                  -0.034229         -0.290027   

   Airline_Trujet  Airline_Vistara  Airline_Vistara Premium economy  \
0       -0.010819        -0.216621                         -0.01874   
1       -0.010819        -0.216621     

In [25]:
### predict one item
data = [['Jet Airways' ,'15/05/2019' ,'Banglore' ,'Delhi','15:15' ,'2h 55m' ,'non-stop']]
columns = ['Airline'	,'Date_of_Journey',	'Source',	'Destination',	'Dep_Time',	'Duration',	'Total_Stops']
# Convert to DataFrame
one_df = pd.DataFrame(data, columns=columns)
pipe_clf.predict(one_df)

array([7686.92228905])

In [22]:
df.head(1)

,Airline,Date_of_Journey,Source,Destination,Dep_Time,Duration,Total_Stops
8417,Jet Airways,15/05/2019,Banglore,Delhi,15:15,2h 55m,non-stop


In [26]:
y_train.head(1)

8417    7229
Name: Price, dtype: int64